#### Loyalty Points Calculation

Importing Libraries

In [1]:
import pandas as pd #library for data manipulation and analysis
import numpy as np #library for adding support for large, multi-dimensional arrays and matrices
import datetime # Library for manipulating dates and time

Inputs

In [2]:

## reading excel_file
excel_file = pd.ExcelFile('Analytics Position Case Study.xlsx') 

## Function to read different sheets and create columns 
def cleaning(ele):
    input_df = pd.read_excel(excel_file, ele)
    #rectifying colmn names and row in dataframe
    new_header = input_df.iloc[2] 
    input_df = input_df[3:] 
    input_df.columns = new_header
    input_df.reset_index(drop=True, inplace=True)
    input_df.rename_axis(None, axis=1, inplace=True)
    input_df.columns = input_df.columns.str.lower()
    # converting datetime from to object to datetime64 format
    input_df['date_time'] = pd.to_datetime(input_df['datetime'])
    input_df['date_new'] = pd.to_datetime(input_df['date_time'].dt.date)
    input_df['session']= np.where(input_df['date_time'].dt.hour< 12, 'S1','S2')
    
    # input_df.rename(columns = {'datetime':'date_time'}, inplace = True)
    return input_df

df_dict ={'games_played':'User Gameplay data','deposit':'Deposit Data','withdrawal':'Withdrawal Data'}

# for key,value in df_dict.items():
#     key +'_df'=cleaning(element)
print(" Below dfs are created for respective data :")
for key,value in df_dict.items():
    exec(f'{key}_df = cleaning(value)')
    print(f'{key}_df')

 Below dfs are created for respective data :
games_played_df
deposit_df
withdrawal_df


In [3]:
### In each table, create session - S1 & S2 based on time
### Union all the dataframe and select distinct of UserID,date,session = Main table
#### Do group by on date and session and take sum of (game played, deposit,withdrwaal)
### (left) Join Main table with the 3 group by table on userid,date & session
### Replace null with 0 in main table
## Finally we will have user Id, Date,Session , GamesPlayed,Deposit ,Withdrwaal

Finding Session and Points

In [4]:
games_played_df['date_time'].dt.hour

# outer joining deposit and withdrawal sheets 
outer_df = pd.merge(deposit_df, withdrawal_df, on=('user id', 'date_new', 'session'), how='outer', suffixes=('_deposit', '_withdrawal'))

# outer joining deposit and withdrawal sheets with games played sheet
final_df = pd.merge(outer_df, games_played_df, on=('user id', 'date_new', 'session'), how='outer', suffixes=('outer', 'game'))

# dropping unecessary columns
final_df.drop(columns=['datetime_deposit', 'date_time_deposit', 'datetime_withdrawal', 'date_time_withdrawal', 'datetime', 'date_time'], inplace=True)

# creating new columns with calaculated sum and count
final_df2 = final_df.groupby(['user id', 'date_new', 'session']).agg(deposit_sum=('amount_deposit', 'sum'), deposit_count=('amount_deposit', 'count'), \
    withdrawal_sum=('amount_withdrawal', 'sum'), withdrawal_count=('amount_withdrawal', 'count'), games_played=('games played', 'sum')).reset_index()

# checking for anything wrong
# final_df2[(final_df2['withdrawal_sum'] == 0) & (final_df2['withdrawal_count'] > 0)]

# difference between count of deposit and withdrawal made
final_df2['difference'] = final_df2.apply(lambda x: x['deposit_count'] - x['withdrawal_count'] if (x['deposit_count'] - x['withdrawal_count']) >0 else 0, axis=1)

# calculation for final points
final_df2['points'] = final_df2['deposit_sum'].multiply(0.01) + final_df2['withdrawal_sum'].multiply(0.005) + final_df2['difference'].multiply(0.001)\
     + final_df2['games_played'].multiply(0.2)

final_df2['user id'] = final_df2['user id'].astype(int)
print(final_df2.info())
print(final_df2.head())

/Users/vaibhavk/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/Users/vaibhavk/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39051 entries, 0 to 39050
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   user id           39051 non-null  int64         
 1   date_new          39051 non-null  datetime64[ns]
 2   session           39051 non-null  object        
 3   deposit_sum       39051 non-null  object        
 4   deposit_count     39051 non-null  int64         
 5   withdrawal_sum    39051 non-null  object        
 6   withdrawal_count  39051 non-null  int64         
 7   games_played      39051 non-null  object        
 8   difference        39051 non-null  int64         
 9   points            39051 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 3.0+ MB
None
   user id   date_new session deposit_sum  deposit_count withdrawal_sum  \
0        0 2022-04-10      S2           0              0              0   
1        0 2022-05-

### Part A

#### Question 1

2nd October Slot S1

In [5]:
df1 = final_df2[(final_df2['date_new'] == '2022-02-10') & (final_df2['session'] == 'S1')]
df1[['user id', 'points']]

,user id,points
23,2,0.4
131,5,55.809
193,6,80.402
234,7,1.0
273,8,1.2
...,...,...
38693,990,14.0
38785,992,8.4
38875,996,2.2
38937,997,1.0


16th October Slot S2

In [6]:
df2 = final_df2[(final_df2['date_new'] == '2022-10-16') & (final_df2['session'] == 'S2')]
df2[['user id', 'points']]

,user id,points
47,2,0.4
156,5,93.618
211,6,0.2
298,8,1.0
360,9,1030.251
...,...,...
38718,990,12.4
38767,991,60.201
38810,992,8.8
38900,996,1.8


18th October Slot S1

In [7]:
df3 = final_df2[(final_df2['date_new'] == '2022-10-18') & (final_df2['session'] == 'S1')]
df3[['user id', 'points']]

,user id,points
50,2,0.4
100,3,0.4
159,5,1.6
250,7,0.6
301,8,1.8
...,...,...
38721,990,14.4
38813,992,9.4
38903,996,41.005
38959,997,0.2


26th October Slot S2

In [8]:
df4 = final_df2[(final_df2['date_new'] == '2022-10-26') & (final_df2['session'] == 'S2')]
df4[['user id', 'points']]

,user id,points
67,2,900.002
176,5,40.804
223,6,0.2
260,7,33.285
318,8,1.4
...,...,...
38830,992,6608.8
38867,995,0.2
38920,996,0.8
38974,997,0.4


#### Question 2

Loyalty Program Winner 

In [9]:
df5 = final_df2.sort_values(by=['points', 'games_played'], ascending=False).head()
df5[['user id', 'points', 'games_played']]

,user id,points,games_played
13467,365,100636.2,168
33150,856,76820.903,303
25492,663,74242.612,212
13470,365,70140.21,114
33174,856,69832.6,188


#### Question 3

Average Deposit Amount

In [10]:
df6 = final_df2['deposit_sum'].mean().round(decimals=2)
print("Average Deposit Amount - {}  \n".format(df6))

Average Deposit Amount - 22809.23  



#### Question 4

Average Deposit Amount per user in a month

In [11]:
df7 = final_df2['deposit_sum'].mean()/final_df2['user id'].count()
print("Average Deposit Amount per user in a month - {}  \n".format(df7))

Average Deposit Amount per user in a month - 0.584088186050309  



#### Question 5

Average number of games played per user

In [12]:
df8 = final_df2['games_played'].mean().round(2)
print("Average number of games played per user - {}  \n".format(df8))

Average number of games played per user - 10.73  



# Part B

Top 50 players

In [13]:
top50_df = final_df2.sort_values(by=['points', 'games_played'], ascending=False)
top50_df[['user id', 'points', 'games_played']].head(50)

,user id,points,games_played
13467,365,100636.2,168
33150,856,76820.903,303
25492,663,74242.612,212
13470,365,70140.21,114
33174,856,69832.6,188
6578,182,62953.2,108
13464,365,58051.53,186
38630,989,56397.0,110
13442,365,56058.86,52
38657,989,51310.854,54


One way of allocating money can be, by giving 40%, 20%, and 10% of the money to the 1st, 2nd, and 3rd player\ respectively. For the remaining players, we can calculate their weighted percentage from the total points of the top 50 players and allocate money apart from the top 3 players accordingly. Giving 40%, 20%, and 10% to the top players is because it will attract more players to play more games.

# Part C

According to me, the formula is unfair because it doesn't prioritize withdrawal of money by the players. If we increase the weightage to the withdrawal money it will encourage more number of players to play more games and deposit more amount of money.